In [1]:
import geopandas as gp
import sys, os
import pandas as pd
sys.path.append('../../dependencies/')
import flopy as fp

In [2]:
sim = fp.mf6.MFSimulation.load(sim_ws = '../../models/sgn_model_3layer/', load_only=['wel'])

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    skipping package ic...
    skipping package npf...
    skipping package rch...
    skipping package riv...
    skipping package drn...
    loading package wel...
    skipping package ghb...
    skipping package obs...
    skipping package oc...
    skipping package sto...
  loading model gwt6...
    loading package disv...
    skipping package ic...
    skipping package adv...
    skipping package dsp...
    skipping package mst...
    skipping package ssm...
    skipping package oc...
    skipping package cnc...
    skipping package obs...
    skipping package gwf6-gwt6...
    skipping package ims6...
    skipping package ims6...


In [3]:
gwf = sim.get_model()

In [4]:
gwf.wel.stress_period_data.data[0]

rec.array([((0, 4736),  0.    , 'well-p1_ln\x00'),
           ((0, 4735),  0.    , 'well-p2_ln\x00'),
           ((0, 4742),  0.    , 'well-p3_ln\x00'),
           ((0, 4739),  0.    , 'well-p4_ln\x00'),
           ((0, 5138),  0.    , 'well-p5_ln\x00'),
           ((0, 5561),  0.    , 'well-p6_ln\x00'),
           ((0, 5991),  0.    , 'well-p7_ln\x00'),
           ((0, 5977),  0.    , 'well-p8_ln\x00'),
           ((0, 5547),  0.    , 'well-p9_ln\x00'),
           ((0, 5544),  0.    , 'well-p10_ln\x00'),
           ((0, 5527),  0.    , 'well-p11_ln\x00'),
           ((0, 10012),  0.    , 'well-r1_ln\x00'),
           ((0, 10025),  0.    , 'well-r2_ln\x00'),
           ((0, 10543),  0.    , 'well-r3_ln\x00'),
           ((0, 10555),  0.    , 'well-r4_ln\x00'),
           ((0, 10568),  0.    , 'well-r5_ln\x00'),
           ((0, 11076),  0.    , 'well-r6_ln\x00'),
           ((0, 11083),  0.    , 'well-r7_ln\x00'),
           ((0, 11096),  0.    , 'well-r8_ln\x00'),
           ((0, 11595

In [5]:
spd = pd.DataFrame.from_records(gwf.wel.stress_period_data.data[0])

In [6]:
gwf.modelgrid.top_botm

array([[113.17749023, 112.95309448, 112.82367706, ...,  99.97387695,
         99.11410522,  98.89460754],
       [ 62.76613235,  62.47645187,  62.20778275, ...,  52.99921036,
         52.95099258,  52.88257599],
       [ 45.47722244,  45.54633331,  45.64457703, ...,  50.73249435,
         50.76224518,  50.76548386],
       [-14.52277756, -14.45366669, -14.35542297, ...,  -9.26750565,
         -9.23775482,  -9.23451614]])

In [7]:
spd['laytop'] = -99999
spd['laybot'] = -99999
spd['layer'] = [i[0] for i in spd.cellid]
spd['cell'] = [i[1] for i in spd.cellid]
spd['laytop'] = gwf.modelgrid.top_botm[spd.layer,spd.cell]
spd['laybot'] = gwf.modelgrid.top_botm[spd.layer+1,spd.cell]
spd['laymidpt'] = (spd.laytop - spd.laybot)/2

In [8]:
spd.iloc[0].boundname

'well-p1_ln\x00'

In [9]:
spd['rootname'] = [i.replace('\x00','').replace("well-","") for i in spd.boundname]

In [10]:
spd.iloc[0].rootname

'p1_ln'

In [11]:
spd.dtypes

cellid        object
q            float64
boundname     object
laytop       float64
laybot       float64
layer          int64
cell           int64
laymidpt     float64
rootname      object
dtype: object

In [12]:
spd.boundname = [f'{i.rstrip()}__{lay}' for i,lay in zip(spd.rootname,spd.layer)]

In [13]:
spd

,cellid,q,boundname,laytop,laybot,layer,cell,laymidpt,rootname
0,"(0, 4736)",0.0000,p1_ln__0,104.773590,69.462875,0,4736,17.655357,p1_ln
1,"(0, 4735)",0.0000,p2_ln__0,104.723152,69.486488,0,4735,17.618332,p2_ln
2,"(0, 4742)",0.0000,p3_ln__0,104.679207,69.459198,0,4742,17.610004,p3_ln
3,"(0, 4739)",0.0000,p4_ln__0,104.633415,69.517403,0,4739,17.558006,p4_ln
4,"(0, 5138)",0.0000,p5_ln__0,104.638702,69.421188,0,5138,17.608757,p5_ln
...,...,...,...,...,...,...,...,...,...
178,"(2, 7590)",-0.0095,19/19__2,51.490337,-8.509663,2,7590,30.000000,19/19
179,"(0, 7057)",-0.0095,19/20__0,104.024193,67.291756,0,7057,18.366219,19/20
180,"(2, 7057)",-0.0095,19/20__2,51.473099,-8.526901,2,7057,30.000000,19/20
181,"(0, 5637)",-0.0095,19/21__0,103.002068,68.493927,0,5637,17.254070,19/21


In [14]:
spd.to_csv('../../data/sgn/wells_with_elev.dat')